In [ ]:
from sklearn.datasets import load_files
import numpy as np
import pandas as pd
import seaborn as sns
from keras.utils import np_utils
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.optimizers import RMSprop, Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import gc

%matplotlib inline

#Data directories
train_directory = '../input/fruits/fruits-360_dataset/fruits-360/Training'
test_directory = '../input/fruits/fruits-360_dataset/fruits-360/Test'

In [ ]:
# Load the data:
train_data = load_files(train_directory)
test_data = load_files(test_directory)

In [ ]:
X_train = np.array(train_data['filenames'][:32000])
#convert file paths to image arrays:
X_train_img_array = []
for file in X_train:
    X_train_img_array.append(img_to_array(load_img(file)))
X_train = np.array(X_train_img_array)
del X_train_img_array


X_test = np.array(test_data['filenames'][:9000])
#convert file paths to image arrays:
X_test_img_array = []
for file in X_test:
    X_test_img_array.append(img_to_array(load_img(file)))
X_test = np.array(X_test_img_array)
del X_test_img_array


y_train = np.array(train_data['target'][:32000])
target_labels = np.array(train_data['target_names'])
y_test = np.array(test_data['target'][:9000])

num_labels = len(np.unique(y_train))

del train_data
del test_data

#images are 255 intensity, so normalize:
X_train = X_train/255
X_test = X_test/255

gc.collect()

print('Shape of training set: ', X_train.shape)
print('Shape of test set: ', X_test.shape)
print('Number of possible labels: ', num_labels)

In [ ]:
#Let's take a look at some of the images

fig1 = plt.figure(figsize = (20,10))
for i, idx in enumerate(np.random.choice(X_train.shape[0], size=18, replace=False)):
    subplt = fig1.add_subplot(3,6,i+1)
    subplt.imshow(X_train[idx])
    label_index = y_train[idx]
    subplt.set_title(target_labels[label_index])


In [ ]:
#Look at target
y_train[:5]

In [ ]:
#Need to one-hot encode the target so it works with the CNN final output softmax:

y_train = np_utils.to_categorical(y_train, num_labels)
y_test = np_utils.to_categorical(y_test, num_labels)

y_train[0]

In [ ]:
#CNN model building time:

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 4, padding = 'same', activation = 'relu', input_shape = (100,100,3)))
model.add(Conv2D(filters = 32, kernel_size = 4, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 64, kernel_size = 4, padding ='same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 4, padding ='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.3))

# model.add(Conv2D(filters = 128, kernel_size = 4, padding ='same', activation = 'relu'))
# model.add(MaxPooling2D(pool_size = 2))
# model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(num_labels, activation = 'softmax'))
model.summary()

In [ ]:
optimizer = Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, amsgrad=False)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Split data into train and validation sets. With size of our data set, we can do ~20% validation

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 0, test_size = 0.2)


In [ ]:
#Fit the model

history = model.fit(X_train,y_train,
        batch_size = 64,
        epochs=20,
        validation_data=(X_val, y_val),
        verbose=1, shuffle=True)

In [ ]:
#Calculate accurate on test set

score = model.evaluate(X_test, y_test, verbose=0)
print('Test % accuracy:', score[1]*100)

In [ ]:
#Look at predictions for test images:

y_pred = model.predict(X_test)

fig2 = plt.figure(figsize=(20, 10))
for i, idx in enumerate(np.random.choice(X_test.shape[0], size=18, replace=False)):
    subplt = fig2.add_subplot(3, 6, i + 1)
    subplt.imshow(X_test[idx])
    pred_index = np.argmax(y_pred[idx])
    true_index = np.argmax(y_test[idx])
    subplt.set_title(f"Actual: {target_labels[true_index]}")
    subplt.text(5, 10, f"Predicted: {target_labels[pred_index]}", color='k', backgroundcolor=("white" if pred_index == true_index else "red"))

In [ ]:
# From https://keras.io/visualization/

# Plot training & validation accuracy values
plt.subplot(2,1,1) 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.subplot(2,1,2) 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#Look at some examples of images we classified wrongly:

wrong_indices =[]
L = len(y_pred)
for i in range(L):
    if np.argmax(y_pred[i]) != np.argmax(y_test[i]):
        wrong_indices.append(i)

print("Size of test set: ", L)
print("Total wrong: ", len(wrong_indices))
print("% Wrong:", 100*len(wrong_indices)/L)
        
fig3 = plt.figure(figsize=(20, 10))
for i, idx in enumerate(np.random.choice(wrong_indices, size=18, replace=False)):
    subplt = fig3.add_subplot(3, 6, i + 1)
    subplt.imshow(X_test[idx])
    pred_index = np.argmax(y_pred[idx])
    true_index = np.argmax(y_test[idx])
    subplt.set_title(f"Actual: {target_labels[true_index]}")
    subplt.text(5, 10, f"Predicted: {target_labels[pred_index]}", color='k', backgroundcolor="red")


It looks like it is hard to classify bananas from the perspective that does not show their curvature, and round red foods, like apple/tomato/nectarine/etc.